In [1]:
import sys
sys.path.append('..')
from utils.paths import get_file_path

import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def estilo(fig):
    fig.update_layout(
        title_font=dict(size=24, family="Arial", color="black"),
        xaxis_tickangle=-45,
        xaxis_tickfont=dict(size=12),
        yaxis_tickfont=dict(size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False,
        margin=dict(t=80, l=40, r=20, b=120),
    )
    return fig

In [3]:
df = pl.scan_csv(
    get_file_path('cleaned', 'exportaciones'),
    low_memory=True
).collect()

### Concentración de mercados: Cálculo del Índice Herfindahl por país

In [4]:
fob_pais = (
    df.group_by("PAIS")
    .agg(pl.col("FOBDOL").sum().alias("FOBDOL"))
)

total_fob = fob_pais.select(pl.col("FOBDOL").sum()).item()

fob_pais = (
    fob_pais.with_columns(
        (pl.col("FOBDOL") / total_fob).alias("participacion")
    )
    .with_columns(
        (pl.col("participacion") ** 2).alias("participacion_cuadrada")
    )
)

hhi_paises = fob_pais.select(pl.col("participacion_cuadrada").sum()).item()
print(f"Índice HHI por país: {hhi_paises:.4f}")

fob_pais_pd = fob_pais.to_pandas()

fob_pais_pd = fob_pais_pd[fob_pais_pd["participacion"] > 0.01]

Índice HHI por país: 0.1190


### Participación de países con más del 1% en las exportaciones totales

In [5]:
fig = px.bar(
    fob_pais_pd.sort_values("participacion", ascending=True),
    x="participacion",
    y="PAIS",
    orientation="h",
    title="Participación de países con más del 1% en las exportaciones totales",
    labels={"participacion": "Participación", "PAIS": "País"},
)
fig = estilo(fig)
fig.update_layout(xaxis_tickformat=".2%", margin=dict(t=60, l=100, r=40, b=40))
fig.show()

> ###### __Finalidad:__ Visualizar los países que representan al menos el 1% del total de las exportaciones, permitiendo identificar los principales destinos y evaluar el grado de diversificación geográfica del comercio exterior.

__Conclusión:__ Las exportaciones están altamente concentradas en Estados Unidos, que supera el 30% de participación, seguido por China y Panamá con niveles notablemente menores. Aunque hay otros países relevantes, la distribución no es equitativa, lo que sugiere una dependencia significativa de unos pocos mercados internacionales.

### Top 10 países con mayor impacto en el índice HHI

In [6]:
top_hhi = fob_pais_pd.sort_values("participacion_cuadrada", ascending=False).head(10)

fig = px.bar(
    top_hhi,
    x="participacion_cuadrada",
    y="PAIS",
    orientation="h",
    title="Top 10 países con mayor impacto en el índice HHI",
    labels={"participacion_cuadrada": "Participación²", "PAIS": "País"},
)
fig = estilo(fig)
fig.update_layout(xaxis_tickformat=".4f")
fig.show()


> ###### __Finalidad:__ Mostrar qué países tienen mayor peso en el índice Herfindahl-Hirschman, es decir, cuáles contribuyen más a la concentración del mercado exportador.

__Conclusión:__ Estados Unidos domina por completo la contribución al HHI, con una participación² que sobrepasa a cualquier otro país por amplio margen. Esto confirma que la concentración del comercio exterior está altamente influenciada por un solo destino, lo que aumenta el riesgo comercial ante cambios en la demanda o políticas de ese país.

### Conclusión general del análisis HHI

El valor del índice HHI y los gráficos asociados indican una concentración moderada-alta en los mercados de exportación. Aunque hay múltiples países con participaciones superiores al 1%, el peso desproporcionado de Estados Unidos compromete la diversificación y puede afectar la resiliencia económica ante choques externos.

### Matriz de origen-destino: principales 10 países destino por exportaciones

In [7]:
# Filtros
top_paises = df.group_by("PAIS").agg(pl.col("FOBDOL").sum()).sort("FOBDOL", descending=True).head(10)
top_departamentos = df.group_by("DEPTO").agg(pl.col("FOBDOL").sum()).sort("FOBDOL", descending=True).head(10)

# Listas
lista_paises = top_paises.get_column("PAIS").to_list()
lista_deptos = top_departamentos.get_column("DEPTO").to_list()

# Dataframe (Filtrado)
df_filtrado = df.filter(
    pl.col("PAIS").is_in(lista_paises) & pl.col("DEPTO").is_in(lista_deptos)
)

In [8]:
matriz_od = df_filtrado.group_by(["DEPTO", "PAIS"]).agg(pl.col("FOBDOL").sum())
matriz_od = matriz_od.to_pandas()
pivot_table = matriz_od.pivot(index="DEPTO", columns="PAIS", values="FOBDOL")

In [9]:
pivot_table.head(10)

PAIS,Brasil,China,Ecuador,España,Estados Unidos,México,Panamá,Países Bajos,Perú,Venezuela
DEPTO,,,,,,,,,,
Antioquia,1.620856e+09,1.146093e+09,7.083376e+09,1.083469e+09,3.786232e+10,4.038426e+09,1.174586e+09,1.224897e+09,3.736686e+09,5.862226e+09
Atlántico,2.215753e+09,1.423387e+09,1.529182e+09,3.556721e+08,7.591262e+09,1.552447e+09,8.103052e+08,4.930430e+08,8.159262e+08,2.687114e+09
Bolívar,6.550403e+09,4.833134e+08,2.631948e+09,6.324314e+08,8.101758e+09,2.098922e+09,7.443294e+08,2.066174e+08,2.563960e+09,1.736384e+09
Cesar,2.603179e+09,9.433077e+08,2.089606e+06,2.352470e+09,7.028595e+09,3.055482e+09,4.598982e+08,8.759409e+09,1.841408e+08,1.063615e+09
Cundinamarca,2.356222e+09,8.867936e+08,4.049349e+09,5.041298e+08,1.351636e+10,1.958416e+09,5.016199e+08,9.362321e+08,2.173870e+09,2.915098e+09
Departamentos Varios,1.651417e+08,2.780270e+10,2.116011e+08,8.081762e+09,6.919903e+10,1.648271e+08,1.908171e+10,2.348256e+09,1.012177e+09,6.444964e+07
La Guajira,1.106700e+09,1.169537e+09,2.357315e+05,2.364059e+09,2.894774e+09,1.585685e+08,5.845563e+07,9.108059e+09,9.656340e+07,2.179442e+09
Petróleo y Derivados,3.301709e+08,2.159004e+09,2.517640e+08,8.113668e+08,5.064935e+10,2.875030e+08,1.307803e+09,2.724573e+08,5.754299e+08,1.440983e+07
Santafé de Bogotá,1.047425e+09,6.599592e+08,7.766527e+09,8.725813e+08,1.806612e+10,2.982073e+09,2.067763e+09,8.320006e+08,3.189457e+09,7.768538e+09


In [10]:
fig = go.Figure(data=go.Heatmap(
    z=pivot_table.values,
    x=pivot_table.columns,
    y=pivot_table.index,
    colorscale="YlGnBu",
    colorbar=dict(title="FOB Dólares")
))

fig.update_layout(
    title="Matriz de origen-destino",
    xaxis_title="País",
    yaxis_title="Departamento",
    xaxis_tickfont=dict(size=12),
    yaxis_tickfont=dict(size=12),
    xaxis=dict(type="category"),
    yaxis=dict(type="category"),
    title_font=dict(size=24, family="Arial", color="black"),
    xaxis_tickangle=-45,
    plot_bgcolor="white",
    paper_bgcolor="white"
)

fig.show()

>###### __Finalidad__:Visualizar la concentración del comercio exterior colombiano desde distintos departamentos hacia los 10 países que concentran el mayor valor exportado en dólares FOB, permitiendo identificar patrones geográficos de exportación y relaciones bilaterales destacada

__Conclusiones__:

+ Estados Unidos sobresale como principal socio comercial en múltiples departamentos, destacándose especialmente en Departamentos Varios, Petróleo y Derivados y Atlántico.

+ Algunos países como Países Bajos o Panamá tienen relaciones relevantes con departamentos específicos, sugiriendo rutas comerciales o productos estratégicos.

+ A pesar de la reducción a solo 10 países, se observa una alta dispersión regional, lo que indica diversidad en los orígenes internos de las exportaciones colombianas.

+ Esta selección permite enfocar el análisis sin perder de vista los actores más relevantes en términos de valor exportado.

###  ¿Qué productos exportan estos países?

In [37]:
# Filtros
grupo = (
    df_filtrado.group_by(["PAIS", "DEPTO", "POSARA"])
    .agg(pl.col("FOBDOL").sum().alias("TOTAL_FOB"))
)

producto_top = (
    grupo.sort(["PAIS", "DEPTO", "TOTAL_FOB"], descending=[False, False, True])
    .group_by(["PAIS", "DEPTO"])
    .agg(pl.col("POSARA").first().alias("POSARA_TOP"), pl.col("TOTAL_FOB").max())
)
# Conversión
producto_top = producto_top.to_pandas()

# Ruta
productos = get_file_path('mappings', 'productos_top')

# Mapping
df_productos = pd.read_csv(productos)
dict_productos = pd.Series(df_productos.PRODUCT.values, index=df_productos.POSARA).to_dict()
producto_top["POSARA_TOP"] = producto_top["POSARA_TOP"].map(dict_productos)
producto_top = producto_top.rename(columns={"POSARA_TOP":"PRODUCTOS"})

In [45]:
def filtro_pais(pais: str):
    if not isinstance(pais, str):
        raise ValueError("El parámetro 'pais' debe ser una cadena de texto.")
    
    pais = pais.strip().upper()
    return producto_top[producto_top["PAIS"].str.strip().str.upper() == pais]

In [46]:
filtro_pais("Estados unidos")

,PAIS,DEPTO,PRODUCTOS,TOTAL_FOB
9,Estados Unidos,Departamentos Varios,Petróleos crudos,5.981488e+10
38,Estados Unidos,Cesar,"Hulla bituminosa, incluso pulverizada, sin agl...",6.805814e+09
52,Estados Unidos,Valle del Cauca,Gemas y metales preciosos,1.229241e+09
57,Estados Unidos,Cundinamarca,Flores y capullos cortados frescos (otras vari...,2.644424e+09
65,Estados Unidos,Antioquia,Gemas y metales preciosos,1.826788e+10
67,Estados Unidos,La Guajira,"Hulla bituminosa, incluso pulverizada, sin agl...",2.862320e+09
69,Estados Unidos,Santafé de Bogotá,Flores y capullos cortados frescos (otras vari...,3.170017e+09
74,Estados Unidos,Bolívar,Petróleos crudos,2.418854e+09
87,Estados Unidos,Atlántico,Polvos de aluminio,3.142712e+09
95,Estados Unidos,Petróleo y Derivados,Petróleos crudos,4.308995e+10


### Evolución de diversificación - Número de países activos por año

In [ ]:
diversificacion = df.group_by("AÑO").agg(pl.col("PAIS").n_unique().alias("num_paises")).sort("AÑO")

In [51]:
diversificacion

AÑO,num_paises
i64,u32
2004,189
2005,185
2006,196
2007,195
2008,204
…,…
2020,212
2021,205
2022,206


In [52]:
fig = px.line(diversificacion,
              x='AÑO',
              y='num_paises',
              title='Número de países activos por año',
              labels={"AÑO": "Año", "num_paises": "Numero de paises"}
              )
fig = estilo(fig)
fig.show()

>###### __Finalidad:__ Analizar si el país ha ampliado o reducido sus mercados internacionales a lo largo del tiempo, evaluando la apertura o concentración geográfica del comercio exterior.

### Dependencia de mercados (Top 5 países y su participación)

In [53]:
top5_paises = fob_pais.sort("FOBDOL", descending=True).head(5)
top5_participacion = top5_paises.with_columns(
    (pl.col("FOBDOL") / total_fob * 100).alias("pct_participacion")
)

In [55]:
print(top5_participacion)

shape: (5, 5)
┌────────────────┬───────────┬───────────────┬────────────────────────┬───────────────────┐
│ PAIS           ┆ FOBDOL    ┆ participacion ┆ participacion_cuadrada ┆ pct_participacion │
│ ---            ┆ ---       ┆ ---           ┆ ---                    ┆ ---               │
│ str            ┆ f64       ┆ f64           ┆ f64                    ┆ f64               │
╞════════════════╪═══════════╪═══════════════╪════════════════════════╪═══════════════════╡
│ Estados Unidos ┆ 2.7012e11 ┆ 0.318327      ┆ 0.101332               ┆ 31.832701         │
│ China          ┆ 4.8965e10 ┆ 0.057703      ┆ 0.00333                ┆ 5.770312          │
│ Panamá         ┆ 4.5006e10 ┆ 0.053038      ┆ 0.002813               ┆ 5.303781          │
│ Venezuela      ┆ 3.7167e10 ┆ 0.0438        ┆ 0.001918               ┆ 4.380031          │
│ Ecuador        ┆ 3.4099e10 ┆ 0.040185      ┆ 0.001615               ┆ 4.018463          │
└────────────────┴───────────┴───────────────┴────────────────────

>###### __Finalidad:__ Identificar qué países representan la mayor proporción de las exportaciones, evaluando el grado de exposición a mercados específicos.